## 1. Introduction

**1.1 Background**

In the United States, there are around 6 million car accidents every year, on average. In 2019 alone, there were an estimated 38,800 deaths due to car accidents. To put it into perspective, that is a little more than 106 deaths per day. These 6 million accidents result in about 3 million people being injured every year. And out of these 3 million injuries, about 2 million of them are permanent injuries. According to a study conducted by the National Highway Traffic Safety Administration in 2014, the "economic and societal harm from motor vehicle crashes" cost $871 billion in one year. These few statistics show just how big of an issue driver safety is in the United States.

**1.2 Problem**

The intention for this project is to analyze car accident data in Seattle in order to provide accurate predictions on the severity of a car accident, given various factors and conditions.

**1.3 Interest**

First and foremost, drivers would be very interested in accurate predictions of the severity of car accidents. Accurate predictions would allow drivers to be more cognizant of their surroundings in more dangerous areas or even avoid these areas entirely. Secondly, several government entities would be very interested in these findings. Accurate predictions would allow the government to improve conditions by enforcing more safety measures in higher risk areas. Emergency personnel and law enforcement could also be stationed closer to these higher risk areas in order to cut down on response time and potentially be able to save a significant amount of lives every year. Lastly, many different private companies would find this information beneficial. Car insurance companies could leverage this data in order to correctly adjust premiums given the severity of potential accidents. Other companies working on new technology to improve driver safety could also use this data to make key business decisions.

## 2. Data

**2.1 Data Source**

The collisions data used for this project was obtained through the CSV file shared in class. The labeled dataset was recorded by the city of Seattle, Washington from the year 2004 through 2020. The data shows the details of car collisions, including the severity, time, and conditions under which the collision occurred. You can download the CSV file [here](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv) and the Metadata file [here](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Metadata.pdf).

**2.2 Data Acquisition**

Import necessary libraries

In [9]:
import pandas as pd
import numpy as np
import requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import random
import json # library to handle JSON files

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge geopy --yes #
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium=0.5.0 --yes #
import folium # map rendering library

print('Libraries Imported')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1h             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

Download data

In [6]:
data_link = 'https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv'
df = pd.read_csv(data_link)
df.shape

(194673, 38)

In [7]:
df.head(5)

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


For reference, the location of these collisions can be visualized geographically.

In [12]:
import folium
from folium.plugins import MarkerCluster

map = folium.Map(location = [47.6062, -122.3321], zoom_start = 12)
markercluster = MarkerCluster().add_to(map)
locations = df[['Y', 'X']][df['Y'].notna()].head(1000)
locations_list = locations.values.tolist()
for point in range(len(locations_list)):
    folium.Marker(locations_list[point]).add_to(markercluster)
    
map

**2.3 Data Cleaning**

View counts of values in different columns to get a feel for the data.

In [13]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [14]:
df['STATUS'].value_counts().to_frame('Count')

,Count
Matched,189786
Unmatched,4887


In [15]:
df['ADDRTYPE'].value_counts().to_frame('Count')

,Count
Block,126926
Intersection,65070
Alley,751


In [16]:
df['SEVERITYCODE'].value_counts().to_frame('Count')

,Count
1,136485
2,58188


In [17]:
df['COLLISIONTYPE'].value_counts().to_frame('Count')

,Count
Parked Car,47987
Angles,34674
Rear Ended,34090
Other,23703
Sideswipe,18609
Left Turn,13703
Pedestrian,6608
Cycles,5415
Right Turn,2956
Head On,2024


In [18]:
df['PERSONCOUNT'].describe()

count    194673.000000
mean          2.444427
std           1.345929
min           0.000000
25%           2.000000
50%           2.000000
75%           3.000000
max          81.000000
Name: PERSONCOUNT, dtype: float64

In [19]:
df['PEDCOUNT'].describe()

count    194673.000000
mean          0.037139
std           0.198150
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           6.000000
Name: PEDCOUNT, dtype: float64

In [20]:
df['PEDCYLCOUNT'].describe()

count    194673.000000
mean          0.028391
std           0.167413
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           2.000000
Name: PEDCYLCOUNT, dtype: float64

In [21]:
df['VEHCOUNT'].describe()

count    194673.000000
mean          1.920780
std           0.631047
min           0.000000
25%           2.000000
50%           2.000000
75%           2.000000
max          12.000000
Name: VEHCOUNT, dtype: float64

In [22]:
df['JUNCTIONTYPE'].value_counts().to_frame('Count')

,Count
Mid-Block (not related to intersection),89800
At Intersection (intersection related),62810
Mid-Block (but intersection related),22790
Driveway Junction,10671
At Intersection (but not related to intersection),2098
Ramp Junction,166
Unknown,9


In [23]:
df['INATTENTIONIND'].value_counts().to_frame('Count')

,Count
Y,29805


In [24]:
df['UNDERINFL'].value_counts().to_frame('Count')

,Count
N,100274
0,80394
Y,5126
1,3995


In [25]:
df['WEATHER'].value_counts().to_frame('Count')

,Count
Clear,111135
Raining,33145
Overcast,27714
Unknown,15091
Snowing,907
Other,832
Fog/Smog/Smoke,569
Sleet/Hail/Freezing Rain,113
Blowing Sand/Dirt,56
Severe Crosswind,25


In [26]:
df['ROADCOND'].value_counts().to_frame('Count')

,Count
Dry,124510
Wet,47474
Unknown,15078
Ice,1209
Snow/Slush,1004
Other,132
Standing Water,115
Sand/Mud/Dirt,75
Oil,64


In [27]:
df['LIGHTCOND'].value_counts().to_frame('Count')

,Count
Daylight,116137
Dark - Street Lights On,48507
Unknown,13473
Dusk,5902
Dawn,2502
Dark - No Street Lights,1537
Dark - Street Lights Off,1199
Other,235
Dark - Unknown Lighting,11


In [28]:
df['SPEEDING'].value_counts().to_frame('Count')

,Count
Y,9333


In [29]:
df['HITPARKEDCAR'].value_counts().to_frame('Count')

,Count
N,187457
Y,7216


Drop the columns which are irrelevant for this project.

In [34]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [36]:
df_1 = df.drop(['OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'], axis = 1)
df_1.head(10)

,SEVERITYCODE,X,Y,ADDRTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,-122.323148,47.703140,Intersection,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),NaN,N,Overcast,Wet,Daylight,NaN
1,1,-122.347294,47.647172,Block,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),NaN,0,Raining,Wet,Dark - Street Lights On,NaN
2,1,-122.334540,47.607871,Block,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),NaN,0,Overcast,Dry,Daylight,NaN
3,1,-122.334803,47.604803,Block,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),NaN,N,Clear,Dry,Daylight,NaN
4,2,-122.306426,47.545739,Intersection,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),NaN,0,Raining,Wet,Daylight,NaN
5,1,-122.387598,47.690575,Intersection,2,0,0,2,2019/04/20 00:00:00+00,4/20/2019 5:42:00 PM,At Intersection (intersection related),NaN,N,Clear,Dry,Daylight,NaN
6,1,-122.338485,47.618534,Intersection,2,0,0,2,2008/12/09 00:00:00+00,12/9/2008,At Intersection (intersection related),NaN,0,Raining,Wet,Daylight,NaN
7,2,-122.320780,47.614076,Intersection,3,0,1,1,2020/04/15 00:00:00+00,4/15/2020 5:47:00 PM,At Intersection (intersection related),NaN,N,Clear,Dry,Daylight,NaN
8,1,-122.335930,47.611904,Block,2,0,0,2,2006/06/15 00:00:00+00,6/15/2006 1:00:00 PM,Mid-Block (not related to intersection),NaN,0,Clear,Dry,Daylight,NaN
9,2,-122.384700,47.528475,Intersection,2,0,0,2,2006/03/20 00:00:00+00,3/20/2006 3:49:00 PM,At Intersection (intersection related),NaN,0,Clear,Dry,Daylight,NaN


Drop location data to work with other features in df_2. Keep location information in df_1 for map visualizations later on.

In [38]:
df_2 = df_1.drop(['X', 'Y', 'INCDATE', 'INCDTTM'], axis = 1)
df_2.head(10)

,SEVERITYCODE,ADDRTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,Intersection,2,0,0,2,At Intersection (intersection related),NaN,N,Overcast,Wet,Daylight,NaN
1,1,Block,2,0,0,2,Mid-Block (not related to intersection),NaN,0,Raining,Wet,Dark - Street Lights On,NaN
2,1,Block,4,0,0,3,Mid-Block (not related to intersection),NaN,0,Overcast,Dry,Daylight,NaN
3,1,Block,3,0,0,3,Mid-Block (not related to intersection),NaN,N,Clear,Dry,Daylight,NaN
4,2,Intersection,2,0,0,2,At Intersection (intersection related),NaN,0,Raining,Wet,Daylight,NaN
5,1,Intersection,2,0,0,2,At Intersection (intersection related),NaN,N,Clear,Dry,Daylight,NaN
6,1,Intersection,2,0,0,2,At Intersection (intersection related),NaN,0,Raining,Wet,Daylight,NaN
7,2,Intersection,3,0,1,1,At Intersection (intersection related),NaN,N,Clear,Dry,Daylight,NaN
8,1,Block,2,0,0,2,Mid-Block (not related to intersection),NaN,0,Clear,Dry,Daylight,NaN
9,2,Intersection,2,0,0,2,At Intersection (intersection related),NaN,0,Clear,Dry,Daylight,NaN


Check for null values in dataframe and deal with them accordingly.

In [39]:
df_2.isnull().sum()

SEVERITYCODE           0
ADDRTYPE            1926
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
JUNCTIONTYPE        6329
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER             5081
ROADCOND            5012
LIGHTCOND           5170
SPEEDING          185340
dtype: int64

For these categorical variables, label any missing values with 'Other' or 'Unknown'.

In [41]:
df_2['ADDRTYPE'] = df_2['ADDRTYPE'].fillna('Other')
df_2['JUNCTIONTYPE'] = df_2['JUNCTIONTYPE'].fillna('Other')
df_2['WEATHER'] = df_2['WEATHER'].fillna('Unknown')
df_2['ROADCOND'] = df_2['ROADCOND'].fillna('Unknown')
df_2['LIGHTCOND'] = df_2['LIGHTCOND'].fillna('Unknown')
df_2.head(50)

,SEVERITYCODE,ADDRTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,Intersection,2,0,0,2,At Intersection (intersection related),NaN,N,Overcast,Wet,Daylight,NaN
1,1,Block,2,0,0,2,Mid-Block (not related to intersection),NaN,0,Raining,Wet,Dark - Street Lights On,NaN
2,1,Block,4,0,0,3,Mid-Block (not related to intersection),NaN,0,Overcast,Dry,Daylight,NaN
3,1,Block,3,0,0,3,Mid-Block (not related to intersection),NaN,N,Clear,Dry,Daylight,NaN
4,2,Intersection,2,0,0,2,At Intersection (intersection related),NaN,0,Raining,Wet,Daylight,NaN
5,1,Intersection,2,0,0,2,At Intersection (intersection related),NaN,N,Clear,Dry,Daylight,NaN
6,1,Intersection,2,0,0,2,At Intersection (intersection related),NaN,0,Raining,Wet,Daylight,NaN
7,2,Intersection,3,0,1,1,At Intersection (intersection related),NaN,N,Clear,Dry,Daylight,NaN
8,1,Block,2,0,0,2,Mid-Block (not related to intersection),NaN,0,Clear,Dry,Daylight,NaN
9,2,Intersection,2,0,0,2,At Intersection (intersection related),NaN,0,Clear,Dry,Daylight,NaN


In [42]:
df_2.isnull().sum()

SEVERITYCODE           0
ADDRTYPE               0
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
JUNCTIONTYPE           0
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER                0
ROADCOND               0
LIGHTCOND              0
SPEEDING          185340
dtype: int64

Replace the missing values in 'INATTENTIONIND' with 0 and change the 'Y' values with 1.

In [43]:
df_2['INATTENTIONIND'].value_counts().to_frame('Count')

,Count
Y,29805


In [44]:
df_2['INATTENTIONIND'] = df_2['INATTENTIONIND'].fillna(0)
df_2['INATTENTIONIND'] = df_2['INATTENTIONIND'].replace('Y', 1)
df_2['INATTENTIONIND'].value_counts().to_frame('Count')

,Count
0,164868
1,29805


Replace the missing values in 'UNDERINFL' with 'UNKNWN', the 0 values with 'N', and the 1 values with 'Y'.

In [45]:
df_2['UNDERINFL'].value_counts().to_frame('Count')

,Count
N,100274
0,80394
Y,5126
1,3995


In [47]:
df_2['UNDERINFL'] = df_2['UNDERINFL'].replace('0', 'N')
df_2['UNDERINFL'] = df_2['UNDERINFL'].replace('1', 'Y')
df_2['UNDERINFL'] = df_2['UNDERINFL'].fillna('UNKNWN')
df_2['UNDERINFL'].value_counts().to_frame('Count')

,Count
N,180668
Y,9121
UNKNWN,4884


Replace the missing values in 'SPEEDING' with 0 and the 'Y' values with 1.

In [48]:
df_2['SPEEDING'].value_counts().to_frame('Count')

,Count
Y,9333


In [49]:
df_2['SPEEDING'] = df_2['SPEEDING'].replace('Y', 1)
df_2['SPEEDING'] = df_2['SPEEDING'].fillna(0)
df_2['SPEEDING'].value_counts().to_frame('Count')

,Count
0.0,185340
1.0,9333


Delete rows where 'WEATHER' variable has too few values to matter.

In [50]:
df_2['WEATHER'].value_counts().to_frame('Count')

,Count
Clear,111135
Raining,33145
Overcast,27714
Unknown,20172
Snowing,907
Other,832
Fog/Smog/Smoke,569
Sleet/Hail/Freezing Rain,113
Blowing Sand/Dirt,56
Severe Crosswind,25


In [52]:
df_2.drop(df_2[(df_2['WEATHER'] == 'Partly Cloudy')].index, inplace = True)
df_2['WEATHER'].value_counts().to_frame('Count')

,Count
Clear,111135
Raining,33145
Overcast,27714
Unknown,20172
Snowing,907
Other,832
Fog/Smog/Smoke,569
Sleet/Hail/Freezing Rain,113
Blowing Sand/Dirt,56
Severe Crosswind,25


Delete rows where 'LIGHTCOND' variable has too few values to matter.

In [53]:
df_2['LIGHTCOND'].value_counts().to_frame('Count')

,Count
Daylight,116135
Dark - Street Lights On,48506
Unknown,18643
Dusk,5902
Dawn,2501
Dark - No Street Lights,1536
Dark - Street Lights Off,1199
Other,235
Dark - Unknown Lighting,11


In [54]:
df_2.drop(df_2[(df_2['LIGHTCOND'] == 'Dark - Unknown Lighting')].index, inplace = True)
df_2['LIGHTCOND'].value_counts().to_frame('Count')

,Count
Daylight,116135
Dark - Street Lights On,48506
Unknown,18643
Dusk,5902
Dawn,2501
Dark - No Street Lights,1536
Dark - Street Lights Off,1199
Other,235


Check if any values in 'ROADCOND' are too minimal to matter.

In [55]:
df_2['ROADCOND'].value_counts().to_frame('Count')

,Count
Dry,124502
Wet,47468
Unknown,20088
Ice,1209
Snow/Slush,1004
Other,132
Standing Water,115
Sand/Mud/Dirt,75
Oil,64


The dataset is now cleaned.

In [56]:
#df_3 is cleaned dataset#
df_3 = df_2
df_3.head(10)

,SEVERITYCODE,ADDRTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,Intersection,2,0,0,2,At Intersection (intersection related),0,N,Overcast,Wet,Daylight,0.0
1,1,Block,2,0,0,2,Mid-Block (not related to intersection),0,N,Raining,Wet,Dark - Street Lights On,0.0
2,1,Block,4,0,0,3,Mid-Block (not related to intersection),0,N,Overcast,Dry,Daylight,0.0
3,1,Block,3,0,0,3,Mid-Block (not related to intersection),0,N,Clear,Dry,Daylight,0.0
4,2,Intersection,2,0,0,2,At Intersection (intersection related),0,N,Raining,Wet,Daylight,0.0
5,1,Intersection,2,0,0,2,At Intersection (intersection related),0,N,Clear,Dry,Daylight,0.0
6,1,Intersection,2,0,0,2,At Intersection (intersection related),0,N,Raining,Wet,Daylight,0.0
7,2,Intersection,3,0,1,1,At Intersection (intersection related),0,N,Clear,Dry,Daylight,0.0
8,1,Block,2,0,0,2,Mid-Block (not related to intersection),0,N,Clear,Dry,Daylight,0.0
9,2,Intersection,2,0,0,2,At Intersection (intersection related),0,N,Clear,Dry,Daylight,0.0


In [57]:
df_3.isnull().sum()

SEVERITYCODE      0
ADDRTYPE          0
PERSONCOUNT       0
PEDCOUNT          0
PEDCYLCOUNT       0
VEHCOUNT          0
JUNCTIONTYPE      0
INATTENTIONIND    0
UNDERINFL         0
WEATHER           0
ROADCOND          0
LIGHTCOND         0
SPEEDING          0
dtype: int64

Ensuring correct datatypes for each column. During cleaning, the 'SPEEDING' column was changed to float64. For consistency, this column was changed to int64.

In [58]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194657 entries, 0 to 194672
Data columns (total 13 columns):
SEVERITYCODE      194657 non-null int64
ADDRTYPE          194657 non-null object
PERSONCOUNT       194657 non-null int64
PEDCOUNT          194657 non-null int64
PEDCYLCOUNT       194657 non-null int64
VEHCOUNT          194657 non-null int64
JUNCTIONTYPE      194657 non-null object
INATTENTIONIND    194657 non-null int64
UNDERINFL         194657 non-null object
WEATHER           194657 non-null object
ROADCOND          194657 non-null object
LIGHTCOND         194657 non-null object
SPEEDING          194657 non-null float64
dtypes: float64(1), int64(6), object(6)
memory usage: 20.8+ MB


In [59]:
df_3['SPEEDING'] = df_3['SPEEDING'].astype('int64')
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194657 entries, 0 to 194672
Data columns (total 13 columns):
SEVERITYCODE      194657 non-null int64
ADDRTYPE          194657 non-null object
PERSONCOUNT       194657 non-null int64
PEDCOUNT          194657 non-null int64
PEDCYLCOUNT       194657 non-null int64
VEHCOUNT          194657 non-null int64
JUNCTIONTYPE      194657 non-null object
INATTENTIONIND    194657 non-null int64
UNDERINFL         194657 non-null object
WEATHER           194657 non-null object
ROADCOND          194657 non-null object
LIGHTCOND         194657 non-null object
SPEEDING          194657 non-null int64
dtypes: int64(7), object(6)
memory usage: 20.8+ MB


The final cleaned dataset has 194,657 rows.

In [60]:
df_3.shape

(194657, 13)